In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dtaidistance import dtw
import itertools
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import os

import random

from SVM_models.models import evaluate_binary_SVM_standard, evaluate_binary_SVM_custom

# import GunPoint dataset from GunPoint/GunPoint_test.txt and GunPoint/GunPoint_train.txt


def load_dataset(file_path):
    with open(file_path) as f:
        lines = f.readlines()
    # Parse each line, separating the label and features
    labels = []
    features = []
    for line in lines:
        values = list(map(float, line.strip().split()))
        labels.append(int(values[0])-1)  # First value is the label, cast to int
        features.append(values[1:])    # Remaining values are features
    # Convert lists to numpy arrays for easier handling
    labels = np.array(labels)
    features = np.array(features)
    # normalize dataset to 0 1
    features = (features - features.min()) / (features.max() - features.min())
    return labels, features

# Load the GunPoint dataset

#dataset_path with join

train_path = os.path.join("GunPoint", "GunPoint_TRAIN.txt")
test_path = os.path.join("GunPoint", "GunPoint_TEST.txt")

train_labels, train_features = load_dataset(train_path)
INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES = load_dataset(test_path)

# split test in validation (50) and test (100)



In [2]:
EXPS = [
    "RANDOM",
    "RANDOM_BEST",
    "BAYESIAN_HAMILTONIAN",
    "BAYESIAN_MODEL",
]

OPT_TIMES = 30

CONFIG = {
    "N_QUBITS": 2,
    "SPARSITY": 0.95,
    "ENCODING": "euler",
}

SEEDS = range(10)
EXP = "RANDOM"


In [3]:
import os
def save_results(EXP, encoding, seed, train_accuracy, val_accuracy, test_accuracy, Hamiltonian_c, hidden_alphas):
    # Create folder structure
    folder_name = f"{EXP}__{encoding}"
    seed_folder = os.path.join("exps_results", folder_name, f"exp_{seed}")
    os.makedirs(seed_folder, exist_ok=True)

    # Save configuration and OPT_TIMES to a text file
    with open(os.path.join(seed_folder, "config.txt"), "w") as f:
        f.write(f"Experiment: {EXP}\n")
        f.write(f"Encoding: {encoding}\n")
        f.write(f"Number of Qubits: {CONFIG['N_QUBITS']}\n")
        f.write(f"Sparsity Coefficient: {CONFIG['SPARSITY']}\n")
        f.write(f"OPT_TIMES: {OPT_TIMES}\n")

    # Save train, validation, and test accuracy
    with open(os.path.join(seed_folder, "train_accuracy.txt"), "w") as f:
        f.write(f"Train Accuracy: {train_accuracy}\n")
    
    with open(os.path.join(seed_folder, "val_accuracy.txt"), "w") as f:
        f.write(f"Validation Accuracy: {val_accuracy}\n")

    with open(os.path.join(seed_folder, "test_accuracy.txt"), "w") as f:
        f.write(f"Test Accuracy: {test_accuracy}\n")

    # Save Hamiltonian_c and hidden_alphas
    with open(os.path.join(seed_folder, "Hamiltonian_c.txt"), "w") as f:
        f.write(str(Hamiltonian_c))

    with open(os.path.join(seed_folder, "hidden_alphas.txt"), "w") as f:
        f.write(str(hidden_alphas))


In [4]:
from SVM_models.QuantumSVM import QuantumSVM
from sklearn.metrics import accuracy_score

In [5]:
for SEED in SEEDS:

    random.seed(SEED)
    test_labels, val_labels, test_features, val_features = train_test_split(INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES, test_size=0.33, random_state=SEED)
    
    quantum_svm = QuantumSVM(n_qubits=CONFIG["N_QUBITS"], sparsity_coefficient=CONFIG["SPARSITY"], embedding_code=CONFIG["ENCODING"])

    quantum_svm.fit(train_features, train_labels)
    y_train = quantum_svm.predict_on_train()
    train_accuracy = accuracy_score(train_labels, y_train)
    print(f"Train Accuracy: {train_accuracy}")

    y_val = quantum_svm.predict(val_features)
    val_accuracy = accuracy_score(val_labels, y_val)
    print(f"Validation Accuracy: {val_accuracy}")

    y_test = quantum_svm.predict(test_features)
    accuracy = accuracy_score(test_labels, y_test)
    print(f"Test Accuracy: {accuracy}")


    # now I want to save the model Hamiltonian_c and hidden_alphas
    print("Saving model...")
    Hamiltonian_c = quantum_svm.Hamiltonian_c
    hidden_alphas = quantum_svm.hidden_alphas

    #print("Hamiltonian_c: ", Hamiltonian_c)
    #print("hidden_alphas: ", hidden_alphas)

    save_results(EXP, CONFIG["ENCODING"], SEED, train_accuracy, val_accuracy, accuracy, Hamiltonian_c, hidden_alphas)


Generating random coefficients for the Hamiltonian...

Generating random alphas...
Non-zero alphas: 8


Building TRAIN_KERNEL: 100%|██████████| 1225/1225 [00:42<00:00, 28.89it/s]


Train Accuracy: 0.66


Building TEST_KERNEL: 100%|██████████| 2500/2500 [01:41<00:00, 24.52it/s]


Validation Accuracy: 0.46


...

Train Accuracy: 0.76


Building TEST_KERNEL: 100%|██████████| 2500/2500 [02:36<00:00, 15.96it/s]


Validation Accuracy: 0.68


Building TEST_KERNEL: 100%|██████████| 5000/5000 [05:11<00:00, 16.06it/s]

Test Accuracy: 0.76
Saving model...
